In [0]:
!git clone https://github.com/IlyaSk10/POS_Tagger_with_using_CNN/
%cd POS_Tagger_with_using_CNN
!unzip SYNTAGRUS_texts.zip
!head syntagrus_full.ud
from data_preparation.converter_from_ud_to_txt import UDConverter

UDConverter.convert_from_conllu("syntagrus_full.ud", "syntagrus_fixed.txt")
!head syntagrus_fixed.txt
text_data= "syntagrus_fixed.txt"

Cloning into 'POS_Tagger_with_using_CNN'...
remote: Enumerating objects: 39, done.
remote: Counting objects: 100% (39/39), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 39 (delta 14), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (39/39), done.
/content/POS_Tagger_with_using_CNN
Archive:  SYNTAGRUS_texts.zip
  inflating: syntagrus_full.ud       
1	Начальник	начальник	NOUN	Animacy=Anim|Case=Nom|Gender=Masc|Number=Sing
2	областного	областной	ADJ	Case=Gen|Degree=Pos|Gender=Neut|Number=Sing
3	управления	управление	NOUN	Animacy=Inan|Case=Gen|Gender=Neut|Number=Sing
4	связи	связь	NOUN	Animacy=Inan|Case=Gen|Gender=Fem|Number=Sing
5	Семен	семен	NOUN	Animacy=Anim|Case=Nom|Gender=Masc|Number=Sing
6	Еремеевич	еремеевич	NOUN	Animacy=Anim|Case=Nom|Gender=Masc|Number=Sing
7	был	быть	VERB	Gender=Masc|Mood=Ind|Number=Sing|Tense=Past|VerbForm=Fin|Voice=Act
8	человек	человек	NOUN	Animacy=Anim|Case=Nom|Gender=Masc|Number=Sing
9	простой	простой	ADJ	Case=Nom|Degree=Pos|Ge

In [0]:
def load_from_file(file):
  with open(file,'r',encoding='utf-8') as f:
    read_file=f.read().split('\n')
  return read_file

In [0]:
dataset=load_from_file(text_data)

In [0]:
import string
dataset = list(filter(None, dataset))
text = ''.join([('' if iteration.split('\t')[2]=='PUNCT' else ' ')+iteration.split('\t')[0] for iteration in dataset[:200]]).strip()
print(text)

Начальник областного управления связи Семен Еремеевич был человек простой, приходил на работу всегда вовремя, здоровался с секретаршей за руку и иногда даже писал в стенгазету заметки под псевдонимом" Муха". В приемной его с утра ожидали посетители,- кое-кто с важными делами, а кое-кто и с такими, которые легко можно было решить в нижестоящих инстанциях, не затрудняя Семена Еремеевича. Однако стиль работы Семена Еремеевича заключался в том, чтобы принимать всех желающих и лично вникать в дело. Приемная была обставлена просто, но по-деловому. У двери стоял стол секретарши, на столе- пишущая машинка с широкой кареткой. В углу висел репродуктор и играло радио для развлечения ожидающих и еще для того, чтобы заглушать голос начальника, доносившийся из кабинета, так_как, бесспорно, среди посетителей могли находиться и случайные люди. Кабинет отличался скромностью, присущей Семену Еремеевичу. В глубине стоял широкий письменный стол с бронзовыми чернильницами и перед ним два кожаных кресла. Сп

In [0]:
def clear_text (dataset):
  clean_dataset=[]
  for sentence in dataset:
    if sentence.split('\t')[0]!='.' and sentence.split('\t')[2]!='PUNCT':
      clean_dataset.append([sentence.split('\t')[0].lower(),sentence.split('\t')[2]])
    elif sentence.split('\t')[0]=='.':
      clean_dataset.append([sentence.split('\t')[0].lower(),sentence.split('\t')[2]])
    else:
      continue
  return clean_dataset

In [0]:
clean_dataset=clear_text(dataset)

In [0]:
print(clean_dataset[:20])

[['начальник', 'NOUN'], ['областного', 'ADJ'], ['управления', 'NOUN'], ['связи', 'NOUN'], ['семен', 'NOUN'], ['еремеевич', 'NOUN'], ['был', 'VERB'], ['человек', 'NOUN'], ['простой', 'ADJ'], ['приходил', 'VERB'], ['на', 'ADP'], ['работу', 'NOUN'], ['всегда', 'ADV'], ['вовремя', 'ADV'], ['здоровался', 'VERB'], ['с', 'ADP'], ['секретаршей', 'NOUN'], ['за', 'ADP'], ['руку', 'NOUN'], ['и', 'CONJ']]


In [0]:
def concatenate_words(dataset):
  concat_words=[]
  concat_labels=[]
  sentence=[]
  labels=[]
  for word in dataset:
    if word[1]!='PUNCT':
      sentence.append(word[0])
      labels.append(word[1])
    else:
      concat_words.append([word for word in sentence])
      concat_labels.append([label for label in labels])
      sentence.clear()
      labels.clear()
  return concat_words,concat_labels

In [0]:
text,labels_sen=concatenate_words(clean_dataset)

In [0]:
clean_dataset=[line for line in clean_dataset if line!='']
MAX_LEN_TOKEN=max(len(line[0]) for line in clean_dataset)
NUMBER_UNIQUE_TOKEN=len(set(line[0] for line in clean_dataset if line[1]!='PUNCT'))
MAX_LEN_SENTENCE=max(len(sentence) for sentence in text)
print('Максимальная длина токена', MAX_LEN_TOKEN)
print('Количество уникальных токенов' , NUMBER_UNIQUE_TOKEN)
print('Максимальная длина предложения' , MAX_LEN_SENTENCE)

Максимальная длина токена 31
Количество уникальных токенов 104251
Максимальная длина предложения 216


In [0]:
labels = ['<NOTAG>'] + sorted({token[1] for token in clean_dataset})
label2id = {label: i+1 for i, label in enumerate(labels)}
print('Метки частей речи' , label2id)

Метки частей речи {'<NOTAG>': 1, 'ADJ': 2, 'ADP': 3, 'ADV': 4, 'CONJ': 5, 'DET': 6, 'H': 7, 'INTJ': 8, 'NOUN': 9, 'NUM': 10, 'PART': 11, 'PRON': 12, 'PUNCT': 13, 'VERB': 14}


In [0]:
from collections import Counter
def build_vocab(text,pad_symbol='<PAD>'):
  sentence=' '.join(line[0].lower() for line in text)
  mydict = dict((j, i+1) for i, j in enumerate(set(sentence)))
  mydict.update({pad_symbol:max(mydict.values())+1})
  return mydict

def counter(dataset,most_freq_symbols):
  if most_freq_symbols is None:
    most_freq_symbols=5
  sentence=' '.join([word[0] for word in dataset])
  return Counter(list(sentence)).most_common(most_freq_symbols)    


In [0]:
print('Символы словаря',build_vocab(text=clean_dataset[:10]),'\n','Наиболее частотные символы',counter(clean_dataset[:10],most_freq_symbols=10))

Символы словаря {'и': 1, 'б': 2, 'г': 3, 'п': 4, 'в': 5, 'с': 6, 'х': 7, ' ': 8, 'е': 9, 'я': 10, 'ы': 11, 'ч': 12, 'т': 13, 'й': 14, 'з': 15, 'н': 16, 'к': 17, 'а': 18, 'л': 19, 'м': 20, 'ь': 21, 'о': 22, 'р': 23, 'д': 24, 'у': 25, '<PAD>': 26} 
 Наиболее частотные символы [(' ', 9), ('е', 9), ('о', 7), ('л', 6), ('и', 6), ('н', 5), ('а', 4), ('с', 4), ('р', 4), ('в', 4)]


In [0]:
vocab=build_vocab(text=clean_dataset)

import numpy as np

A=np.zeros(shape=(len(text[:40000]),MAX_LEN_SENTENCE,MAX_LEN_TOKEN))
B=np.zeros(shape=(len(text[:40000]),MAX_LEN_SENTENCE))
for i in range(len(text[:40000])):
  for j,word in enumerate(text[i]):
    conv=[number for letter in word for symbol,number in vocab.items() if letter==symbol]
    conv1=[number for symbol,number in label2id.items() if labels_sen[i][j]==symbol]
    np.put(A[i,j],[k for k in range(len(word))],conv)
    np.put(B[i],j,conv1)


In [0]:
A[1][:5]

array([[ 6.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.],
       [63., 51., 24., 59., 16., 46.,  7., 55.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.],
       [59., 72.,  7.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.],
       [73.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.],
       [70., 18., 51., 47.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.]])

In [0]:
B[1][:5]

array([ 3.,  9., 12.,  3.,  9.])

In [0]:
def add_pad(words2id,text,pad_symbol,max_len_token,add_boundary=True):
  mat_with_boundary=np.zeros(shape=(len(text),MAX_LEN_SENTENCE,MAX_LEN_TOKEN+1),dtype=int)
  for i in range(len(text)):
    for j in range(len(text[i])):
      if add_boundary==True:
        mat_with_boundary[i,j]=np.insert(words2id[i,j],0,pad_symbol)
      else:
        continue
  return words2id if (add_boundary==False) else mat_with_boundary

In [0]:
A=add_pad(A,text[:40000],pad_symbol=0,max_len_token=MAX_LEN_TOKEN,add_boundary=True)

In [0]:
A[1,:10]

array([[ 0,  6,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0, 63, 51, 24, 59, 16, 46,  7, 55,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0, 59, 72,  7,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0, 73,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0, 70, 18, 51, 47,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  7, 23, 24, 53, 47, 34, 24,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0, 63,  7, 73, 59, 18, 24, 18, 59, 34, 24,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0

In [0]:
B[1]

array([ 3.,  9., 12.,  3.,  9., 14.,  9., 12.,  3.,  2.,  9.,  5., 12.,
       11.,  3.,  6.,  6.,  4.,  4., 14., 14.,  3.,  2.,  9., 11., 14.,
        9.,  9.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0

In [0]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(A, B, test_size=0.33, random_state=42)

from keras.utils import to_categorical

y_train_cat=to_categorical(y_train, num_classes=len(label2id.values())+1)
y_test_cat=to_categorical(y_test, num_classes=len(label2id.values())+1)

Using TensorFlow backend.


In [0]:
import torch
from torch import nn

class StackedConv1d(nn.Module):
  def __init__(self,features_num,layers_n=1,kernel_size=3,conv_layer=nn.Conv1d,dropout=0.0):
    super().__init__()
    layers=[]
    for _ in range(layers_n):
      layers.append(nn.Sequential(conv_layer(features_num,features_num,kernel_size,padding=kernel_size//2),
                                  nn.Dropout(dropout),
                                  nn.LeakyReLU()))
      self.layers=nn.ModuleList(layers)

  # simple ResNet
  def forward(self,x):
    for layer in self.layers:
      x=x+layer(x)
    return x

In [0]:
class SingleTokenPOSTagger(nn.Module):
    def __init__(self, vocab_size, labels_num, embedding_size=32, **kwargs):
        super().__init__()
        self.char_embeddings = nn.Embedding(vocab_size, embedding_size, padding_idx=0)
        self.backbone = StackedConv1d(embedding_size, **kwargs)
        self.global_pooling = nn.AdaptiveMaxPool1d(1)
        self.out = nn.Linear(embedding_size, labels_num)
        self.labels_num = labels_num
    
    def forward(self, tokens):
        """tokens - BatchSize x MaxSentenceLen x MaxTokenLen"""
        batch_size, max_sent_len, max_token_len = tokens.shape
        tokens_flat = tokens.view(batch_size * max_sent_len, max_token_len)
        
        char_embeddings = self.char_embeddings(tokens_flat)  # BatchSize*MaxSentenceLen x MaxTokenLen x EmbSize
        char_embeddings = char_embeddings.permute(0, 2, 1)  # BatchSize*MaxSentenceLen x EmbSize x MaxTokenLen
        
        features = self.backbone(char_embeddings)
        
        global_features = self.global_pooling(features).squeeze(-1)  # BatchSize*MaxSentenceLen x EmbSize
        
        logits_flat = self.out(global_features)  # BatchSize*MaxSentenceLen x LabelsNum
        logits = logits_flat.view(batch_size, max_sent_len, self.labels_num)  # BatchSize x MaxSentenceLen x LabelsNum
        logits = logits.permute(0, 2, 1)  # BatchSize x LabelsNum x MaxSentenceLen
        return logits

In [0]:
single_token_model=SingleTokenPOSTagger(len(vocab),len(label2id),embedding_size=64,layers_n=3,kernel_size=3,dropout=0.3)
print("Количество параметров", sum(np.product(t.shape) for t in single_token_model.parameters()))

Количество параметров 43342


In [0]:
from torch.utils.data import DataLoader
import copy
import datetime
import random
import traceback

import numpy as np
import torch
from torch.utils.data import DataLoader


def init_random_seed(value=0):
    random.seed(value)
    np.random.seed(value)
    torch.manual_seed(value)
    torch.cuda.manual_seed(value)
    torch.backends.cudnn.deterministic = True


def copy_data_to_device(data, device):
    if torch.is_tensor(data):
        return data.to(device)
    elif isinstance(data, (list, tuple)):
        return [copy_data_to_device(elem, device) for elem in data]
    raise ValueError('Недопустимый тип данных {}'.format(type(data)))


def print_grad_stats(model):
    mean = 0
    std = 0
    norm = 1e-5
    for param in model.parameters():
        grad = getattr(param, 'grad', None)
        if grad is not None:
            mean += grad.data.abs().mean()
            std += grad.data.std()
            norm += 1
    mean /= norm
    std /= norm
    print(f'Mean grad {mean}, std {std}, n {norm}')

def train_eval_loop(model, train_dataset, val_dataset, criterion,
                    lr=1e-4, epoch_n=10, batch_size=32,
                    device=None, early_stopping_patience=10, l2_reg_alpha=0,
                    max_batches_per_epoch_train=10000,
                    max_batches_per_epoch_val=1000,
                    data_loader_ctor=DataLoader,
                    optimizer_ctor=None,
                    lr_scheduler_ctor=None,
                    shuffle_train=True,
                    dataloader_workers_n=0):
    """
    Цикл для обучения модели. После каждой эпохи качество модели оценивается по отложенной выборке.
    :param model: torch.nn.Module - обучаемая модель
    :param train_dataset: torch.utils.data.Dataset - данные для обучения
    :param val_dataset: torch.utils.data.Dataset - данные для оценки качества
    :param criterion: функция потерь для настройки модели
    :param lr: скорость обучения
    :param epoch_n: максимальное количество эпох
    :param batch_size: количество примеров, обрабатываемых моделью за одну итерацию
    :param device: cuda/cpu - устройство, на котором выполнять вычисления
    :param early_stopping_patience: наибольшее количество эпох, в течение которых допускается
        отсутствие улучшения модели, чтобы обучение продолжалось.
    :param l2_reg_alpha: коэффициент L2-регуляризации
    :param max_batches_per_epoch_train: максимальное количество итераций на одну эпоху обучения
    :param max_batches_per_epoch_val: максимальное количество итераций на одну эпоху валидации
    :param data_loader_ctor: функция для создания объекта, преобразующего датасет в батчи
        (по умолчанию torch.utils.data.DataLoader)
    :return: кортеж из двух элементов:
        - среднее значение функции потерь на валидации на лучшей эпохе
        - лучшая модель
    """
    if device is None:
        device = 'cuda' if torch.cuda.is_available() else 'cpu'
    device = torch.device(device)
    model.to(device)

    if optimizer_ctor is None:
        optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=l2_reg_alpha)
    else:
        optimizer = optimizer_ctor(model.parameters(), lr=lr)

    if lr_scheduler_ctor is not None:
        lr_scheduler = lr_scheduler_ctor(optimizer)
    else:
        lr_scheduler = None

    train_dataloader = data_loader_ctor(train_dataset, batch_size=batch_size, shuffle=shuffle_train,
                                        num_workers=dataloader_workers_n)
    val_dataloader = data_loader_ctor(val_dataset, batch_size=batch_size, shuffle=False,
                                      num_workers=dataloader_workers_n)

    best_val_loss = float('inf')
    best_epoch_i = 0
    best_model = copy.deepcopy(model)

    for epoch_i in range(epoch_n):
        try:
            epoch_start = datetime.datetime.now()
            print('Эпоха {}'.format(epoch_i))

            model.train()
            mean_train_loss = 0
            train_batches_n = 0
            for batch_i, (batch_x, batch_y) in enumerate(train_dataloader):
                if batch_i > max_batches_per_epoch_train:
                    break

                batch_x = copy_data_to_device(batch_x, device)
                batch_y = copy_data_to_device(batch_y, device)

                pred = model(batch_x)
                loss = criterion(pred, batch_y)

                model.zero_grad()
                loss.backward()

                optimizer.step()

                mean_train_loss += float(loss)
                train_batches_n += 1

            mean_train_loss /= train_batches_n
            print('Эпоха: {} итераций, {:0.2f} сек'.format(train_batches_n,
                                                           (datetime.datetime.now() - epoch_start).total_seconds()))
            print('Среднее значение функции потерь на обучении', mean_train_loss)



            model.eval()
            mean_val_loss = 0
            val_batches_n = 0

            with torch.no_grad():
                for batch_i, (batch_x, batch_y) in enumerate(val_dataloader):
                    if batch_i > max_batches_per_epoch_val:
                        break

                    batch_x = copy_data_to_device(batch_x, device)
                    batch_y = copy_data_to_device(batch_y, device)

                    pred = model(batch_x)
                    loss = criterion(pred, batch_y)

                    mean_val_loss += float(loss)
                    val_batches_n += 1

            mean_val_loss /= val_batches_n
            print('Среднее значение функции потерь на валидации', mean_val_loss)

            if mean_val_loss < best_val_loss:
                best_epoch_i = epoch_i
                best_val_loss = mean_val_loss
                best_model = copy.deepcopy(model)
                print('Новая лучшая модель!')
            elif epoch_i - best_epoch_i > early_stopping_patience:
                print('Модель не улучшилась за последние {} эпох, прекращаем обучение'.format(
                    early_stopping_patience))
                break

            if lr_scheduler is not None:
                lr_scheduler.step(mean_val_loss)

            print()
        except KeyboardInterrupt:
            print('Досрочно остановлено пользователем')
            break
        except Exception as ex:
            print('Ошибка при обучении: {}\n{}'.format(ex, traceback.format_exc()))
            break

    return best_val_loss, best_model

In [0]:
from torch.utils.data import TensorDataset

X_train=torch.from_numpy(X_train).type(torch.LongTensor)
y_train=torch.from_numpy(y_train).type(torch.LongTensor)
X_test=torch.from_numpy(X_test).type(torch.LongTensor)
y_test=torch.from_numpy(y_test).type(torch.LongTensor)

train_dataset=TensorDataset(X_train,y_train)
test_dataset=TensorDataset(X_test,y_test)

In [0]:
from torch.nn import functional as F

(best_val_loss,
 best_single_token_model) = train_eval_loop(single_token_model,
                                            train_dataset,
                                            test_dataset,
                                            F.cross_entropy,
                                            lr=5e-3,
                                            epoch_n=10,
                                            batch_size=64,
                                            device='cuda',
                                            early_stopping_patience=5,
                                            max_batches_per_epoch_train=500,
                                            max_batches_per_epoch_val=100,
                                            lr_scheduler_ctor=lambda optim: torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=2,
                                                                                                                       factor=0.5,
                                                                                                                       verbose=True))

Эпоха 0
Эпоха: 105 итераций, 28.84 сек
Среднее значение функции потерь на обучении 0.20677193126508167
Среднее значение функции потерь на валидации 0.10495220139049567
Новая лучшая модель!

Эпоха 1
Эпоха: 105 итераций, 28.62 сек
Среднее значение функции потерь на обучении 0.06591090297415143
Среднее значение функции потерь на валидации 0.07438985401621231
Новая лучшая модель!

Эпоха 2
Эпоха: 105 итераций, 28.57 сек
Среднее значение функции потерь на обучении 0.057468069415716895
Среднее значение функции потерь на валидации 0.07220508322979395
Новая лучшая модель!

Эпоха 3
Эпоха: 105 итераций, 28.53 сек
Среднее значение функции потерь на обучении 0.05236339278164364
Среднее значение функции потерь на валидации 0.06476892960759309
Новая лучшая модель!

Эпоха 4
Эпоха: 105 итераций, 28.53 сек
Среднее значение функции потерь на обучении 0.05002376501049314
Среднее значение функции потерь на валидации 0.06100103373710926
Новая лучшая модель!

Эпоха 5
Эпоха: 105 итераций, 28.53 сек
Среднее зн